In [1]:
import os
import numpy as np
import cv2  
import glob 
from matplotlib import pyplot as plt
import joblib

In [2]:
#Changing Current Working Directory;
path = r"E:\Dataset\Alzheimer_s Dataset"
def change_dir(path):
    retval = os.getcwd()    # Check current working directory.
    print("Current working directory ", retval) 
    os.chdir( path )   # Now change the directory.
    retval = os.getcwd() # Check current working directory.
    print ("Directory changed successfully ",retval) 
change_dir(path)

Current working directory  C:\Users\iamne\JupyterNotebookServer\dev\ProjAlzhiemer
Directory changed successfully  E:\Dataset\Alzheimer_s Dataset


In [3]:
#saving model 
def save_mod(model,Model_name):
    change_dir(r"C:\Users\iamne\JupyterNotebookServer\dev\ProjAlzhiemer")
    Model_name+='.sav'
    joblib.dump(model,Model_name)

In [4]:
# reading images from directory

def load_image(path):
    lst=[]
    for image_path in glob.glob(path):
        img = cv2.imread(image_path,0)
        img = img.reshape(1,36608)
        lst.append(img[0])
    return lst
    
temp = path + r"\train\MildDemented\*.jpg"
Mild_train = load_image(temp)
print("Mild = ",len(Mild_train))
temp = path + r"\train\ModerateDemented\*.jpg"
Mod_train = load_image(temp)
print("Mod = ",len(Mod_train))
temp = path + r"\train\NonDemented\*.jpg"
Non_train = load_image(temp)
print("Non = ",len(Non_train))
temp = path + r"\train\VeryMildDemented\*.jpg"
VMild_train = load_image(temp)
print("VMild = ",len(VMild_train))

Mild =  717
Mod =  52
Non =  2560
VMild =  1792


In [ ]:

    
temp = path + r"\test\MildDemented\*.jpg"
Mild_test = load_image(temp)
print("Mild = ",len(Mild_test))
temp = path + r"\test\ModerateDemented\*.jpg"
Mod_test = load_image(temp)
print("Mod = ",len(Mod_test))
temp = path + r"\test\NonDemented\*.jpg"
Non_test = load_image(temp)
print("Non = ",len(Non_test))
temp = path + r"\test\VeryMildDemented\*.jpg"
VMild_test = load_image(temp)
print("VMild = ",len(VMild_test))

In [9]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

In [10]:
train_x = Mild_train+VMild_train+Mod_train+Non_train
train_y = []

def ohe_ml(*lst):
    train_y = []
    en =1
    for j in range(0,len(lst)):
        for i in range(0,len(lst[j])):
            train_y.append(en)
        en*=10
    return train_y
train_y= ohe_ml( Mild_train,VMild_train,Mod_train,Non_train)

In [ ]:
test_x = Mild_test+VMild_test+Mod_test+Non_test
test_y = []

def ohe_ml(*lst):
    train_y = []
    en =1
    for j in range(0,len(lst)):
        for i in range(0,len(lst[j])):
            train_y.append(en)
        en*=10
    return train_y
test_y= ohe_ml( Mild_test,VMild_test,Mod_test,Non_test)

In [11]:
print("train\n")
print(len(train_x),len(train_y))
print(train_x[0].shape)
loc = 732
print(train_x[loc],train_y[loc])
#--------------------------------------
print("test\n")
print(len(test_x),len(test_y))
print(test_x[0].shape)
loc = 340
print(test_x[loc],test_y[loc])

5121 5121
(36608,)
[0 0 0 ... 0 0 0] 10


In [12]:
models = list()

decision_tree = Pipeline([('m', KNeighborsClassifier())])
models.append(('knn', decision_tree))

randomforest = Pipeline([('m', RandomForestClassifier())])
models.append(('randomforest', randomforest))

gnb = Pipeline([('m', GaussianNB())])
models.append(('GNB', gnb))
# define the voting ensemble
lr = LogisticRegression()
stack_model = StackingClassifier( estimators = models,final_estimator = lr)

In [13]:
stack_model

StackingClassifier(estimators=[('knn',
                                Pipeline(steps=[('m',
                                                 KNeighborsClassifier())])),
                               ('randomforest',
                                Pipeline(steps=[('m',
                                                 RandomForestClassifier())])),
                               ('GNB', Pipeline(steps=[('m', GaussianNB())]))],
                   final_estimator=LogisticRegression())

In [14]:
models

[('knn', Pipeline(steps=[('m', KNeighborsClassifier())])),
 ('randomforest', Pipeline(steps=[('m', RandomForestClassifier())])),
 ('GNB', Pipeline(steps=[('m', GaussianNB())]))]

In [15]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3)
n_scores = cross_val_score(ensemble, train_x, train_y, scoring='accuracy', cv=cv, n_jobs=6)

In [35]:
mean = np.mean(n_scores)
print("mean = ", mean)

mean =  0.9760232119422572


In [16]:
ens = stack_model.fit(train_x,train_y)

In [18]:
ser=ens.score(test_x,test_y)
print(ser)

0.9462995508689709


In [ ]:
save_mod(e,"test_tf_df")